# Project Title:
**AI Guardian: An AI-Powered Consumer Protection Assistant**

## Project Overview:
The AI Guardian project aims to develop an AI-powered assistant designed to help consumers navigate the digital marketplace safely and confidently. The assistant will focus on providing reliable information, detecting scams and misinformation, and offering support for consumer rights and redress mechanisms.

## Key Features:
1. **Trustworthy Information Source**:
   - **Verified Answers**: Use natural language processing (NLP) to provide consumers with verified and citation-backed answers to their queries.
   - **Bias Detection**: Implement algorithms to detect and mitigate regional or cultural biases in information provided.

2. **Scam and Misinformation Detection**:
   - **Real-time Scam Alerts**: Use machine learning to identify and alert users about potential scams in real-time, whether in emails, advertisements, or online shopping platforms.
   - **Misinformation Filtering**: Analyze content from various sources and flag misinformation, providing users with accurate and verified alternatives.

3. **Consumer Rights Education**:
   - **Interactive Learning Modules**: Offer interactive and engaging modules to educate consumers about their rights, especially in relation to AI and digital services.
   - **Guided Redress Mechanisms**: Provide step-by-step guidance on how consumers can seek redress for issues with AI technologies or digital services.

4. **Personalized Recommendations and Alerts**:
   - **Custom Alerts**: Based on user preferences and past behavior, the assistant can offer personalized alerts about new scams or critical updates in consumer protection laws.
   - **Safe Shopping Recommendations**: Recommend safe and reliable online marketplaces and vendors based on AI analysis of consumer reviews and ratings.

## Import Libraries

In [1]:
import os 
import dotenv
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from typing import Dict
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

## Load API KEYS

In [2]:
dotenv.load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Initializing Chat Model
Let's initialize the chat model which will serve as the chatbot's brain:

In [3]:
Model = 'llama3-8b-8192'

chat = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name=Model)

## Invoking The Model
If we invoke our chat model, the output is an AIMessage:

In [4]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Large Language Models (LLMs) have revolutionized the field of natural language processing (NLP) by enabling applications such as language translation, text summarization, and chatbots. However, traditional LLMs often suffer from high latency, which can be a significant limitation in many real-world applications. Low latency LLMs, on the other hand, offer several advantages that make them crucial for various use cases. Here are some reasons why low latency LLMs are important:\n\n1. **Real-time processing**: Low latency LLMs enable real-time processing of user input, which is essential for applications like chatbots, virtual assistants, and live language translation. This allows for a more seamless and responsive user experience.\n2. **Improved user engagement**: Fast response times can significantly improve user engagement and satisfaction. Low latency LLMs can process user input quickly, reducing the likelihood of user frustration and abandonment.\n3. **Enhanced deci

In [5]:
chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

AIMessage(content='The translation of "I love programming" in French is:\n\nJ\'adore le programmation.\n\nHere\'s a breakdown of the translation:\n\n* "I" is translated to "J\'" (the subject pronoun in French)\n* "love" is translated to "adore" (which is a stronger form of "aimer", meaning "to love")\n* "programming" is translated to "le programmation" (with the definite article "le" because "programmation" is a masculine noun)\n\nSo, "J\'adore le programmation" is the correct translation of "I love programming" in French!', response_metadata={'token_usage': {'completion_time': 0.154, 'completion_tokens': 128, 'prompt_time': 0.009, 'prompt_tokens': 22, 'queue_time': None, 'total_time': 0.163, 'total_tokens': 150}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-39a0da02-dd64-48fd-a521-9de808abf4fb-0')

## Model State
The model on its own does not have any concept of state. For example, if you ask a followup question:

In [6]:
chat.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content="I apologize, but this conversation just started. I haven't said anything yet. I'm here to help answer your questions and provide information. What would you like to talk about?", response_metadata={'token_usage': {'completion_time': 0.04, 'completion_tokens': 36, 'prompt_time': 0.004, 'prompt_tokens': 16, 'queue_time': None, 'total_time': 0.044, 'total_tokens': 52}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-94cc8481-7de8-413e-bf77-0b438460185e-0')

We can see that it doesn't take the previous conversation turn into context, and cannot answer the question.

To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:

In [7]:
chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        ),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content="What did you just say?"),
    ]
)

AIMessage(content='I translated the sentence "I love programming" from English to French. The translation is:\n\nJ\'adore la programmation.\n\nHere\'s a breakdown of the translation:\n\n* "I" is translated to "J\'" (note the accent on the "J" to indicate the contraction)\n* "love" is translated to "adore" (which means "to love" or "to adore")\n* "programming" is translated to "la programmation" (using the definite article "la" to indicate that it\'s a singular noun)\n\nSo, the entire sentence "J\'adore la programmation" means "I love programming" in French!', response_metadata={'token_usage': {'completion_time': 0.158, 'completion_tokens': 133, 'prompt_time': 0.014, 'prompt_tokens': 46, 'queue_time': None, 'total_time': 0.17200000000000001, 'total_tokens': 179}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7f6d9d7-3d3d-482f-a952-a2e092b3019f-0')

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationally.

## Prompt templates
Let's define a prompt template to make formatting a bit easier. We can create a chain by piping it into the model:

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

The MessagesPlaceholder above inserts chat messages passed into the chain's input as chat_history directly into the prompt. Then, we can invoke the chain like this:

In [9]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I translated the sentence "I love programming" from English to French. The translation is: "J\'adore la programmation".', response_metadata={'token_usage': {'completion_time': 0.032, 'completion_tokens': 27, 'prompt_time': 0.02, 'prompt_tokens': 67, 'queue_time': None, 'total_time': 0.052000000000000005, 'total_tokens': 94}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2c7c5c4-94e9-487c-ac5f-e5d8bacf7acd-0')

## Message history
As a shortcut for managing the chat history, we can use a MessageHistory class, which is responsible for saving and loading chat messages. There are many built-in message history integrations that persist messages to a variety of databases, but for this quickstart we'll use a in-memory, demo message history called ChatMessageHistory.

In [10]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

Once we do that, we can pass the stored messages directly into our chain as a parameter:

In [11]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

AIMessage(content='The translation of the sentence "I love programming" from English to French is:\n\nJ\'adore le programmation.\n\nLet me know if you need any further assistance!', response_metadata={'token_usage': {'completion_time': 0.038, 'completion_tokens': 34, 'prompt_time': 0.014, 'prompt_tokens': 59, 'queue_time': None, 'total_time': 0.052, 'total_tokens': 93}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-3d8e2fdc-b9cd-4b55-8590-8b7e951c126f-0')

In [12]:
demo_ephemeral_chat_history.add_ai_message(response)

demo_ephemeral_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_ephemeral_chat_history.messages})

AIMessage(content='I translated the sentence "I love programming" from English to French. The French translation is "J\'adore le programmation".', response_metadata={'token_usage': {'completion_time': 0.031, 'completion_tokens': 27, 'prompt_time': 0.025, 'prompt_tokens': 109, 'queue_time': None, 'total_time': 0.056, 'total_tokens': 136}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-629f4acd-941b-4652-a030-b157cfaf9401-0')

**And now we have a basic chatbot!**

```While this chain can serve as a useful chatbot on its own with just the model's internal knowledge, it's often useful to introduce some form of retrieval-augmented generation, or RAG for short, over domain-specific knowledge to make our chatbot more focused. We'll cover this next.```

## Retrievers
We can set up and use a Retriever to pull domain-specific knowledge for our chatbot. To show this, let's expand the simple chatbot we created above to be able to answer questions about LangSmith.

We'll use the **Consumer Day and related website** as source material and store it in a vectorstore for later retrieval. Note that this example will gloss over some of the specifics around parsing and storing a data source.

To go Deeper - ```https://python.langchain.com/v0.1/docs/use_cases/question_answering/```

In [13]:
loader = WebBaseLoader("https://unctad.org/topic/competition-and-consumer-protection/un-guidelines-for-consumer-protection")
data = loader.load()

Next, we split it into smaller chunks that the LLM's context window can handle and store it in a vector database:

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

Then we embed and store those chunks in a vector database:

### Creating and Saving and Loading the Embeddings

In [15]:
embedding = OpenAIEmbeddings()
persist_dir = 'Embeddings'

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_dir)

And finally, let's create a retriever from our initialized vectorstore:

In [16]:
# Load the vector store
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# Define the retriever
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("Summarise the Guidelines for Consumer Protection")

docs

[Document(page_content='Download:\n\nThe United Nations Guidelines for Consumer Protection\n\t| English | French | Spanish | Arabic | Chinese | Russian |\n\n\n\n\n\n\n\n\n\nRight navigation\n\n\nCompetition and Consumer Protection\n\n\nIntergovernmental Group of Experts on Competition Law and Policy\n\n\nIntergovernmental Group of Experts on Consumer Protection Law and Policy\n\n\nVoluntary Peer Review of Competition Law and Policy\n\n\nVoluntary Peer Review of Consumer Protection Law and Policy', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and regulations that are suitable to their own economic and social and environmental circumstances, as well as promoting international en

We can see that invoking the retriever above results in some parts of the UN guiedins for Consumer Protection document that contain information about testing that our chatbot can use as context when answering questions.

## Handling documents
Let's modify our previous prompt to accept documents as context. We'll use a create_stuff_documents_chain helper function to "stuff" all of the input documents into the prompt, which also conveniently handles formatting. We use the ChatPromptTemplate.from_messages method to format the message input we want to pass to the model, including a MessagesPlaceholder where chat history messages will be directly injected:

In [17]:
# chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

We can invoke this document_chain with the raw documents we retrieved above:

In [18]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("Summarise the Guidelines for Consumer Protection")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

"The United Nations Guidelines for Consumer Protection aim to promote fair and transparent business practices, protect consumers' rights, and ensure their safety and well-being. The guidelines cover various aspects of consumer protection, including:\n\n1. Information and Education: Providing accurate and timely information to consumers, as well as promoting education and awareness about consumer rights and responsibilities.\n2. Protection from Unfair Practices: Prohibiting unfair or deceptive business practices, such as false advertising, bait-and-switch tactics, and other forms of consumer exploitation.\n3. Product Safety and Quality: Ensuring that products meet safety and quality standards, and that manufacturers and suppliers are held accountable for any defects or harm caused by their products.\n4. Consumer Redress: Providing mechanisms for consumers to seek redress for any harm or injury caused by a business or product, including compensation for damages and other forms of relief.

Awesome! We see an answer synthesized from information in the input documents.

## Creating a retrieval chain
Next, let's integrate our retriever into the chain. Our retriever should retrieve information relevant to the last message we pass in from the user, so we extract it and use that as input to fetch relevant docs, which we add to the current chain as context. We pass context plus the previous messages into our document chain to generate a final answer.

We also use the RunnablePassthrough.assign() method to pass intermediate steps through at each invocation. Here's what it looks like:

In [19]:
def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [20]:
response = retrieval_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

response

{'messages': [HumanMessage(content='Summarise the Guidelines for Consumer Protection')],
 'context': [Document(page_content='Download:\n\nThe United Nations Guidelines for Consumer Protection\n\t| English | French | Spanish | Arabic | Chinese | Russian |\n\n\n\n\n\n\n\n\n\nRight navigation\n\n\nCompetition and Consumer Protection\n\n\nIntergovernmental Group of Experts on Competition Law and Policy\n\n\nIntergovernmental Group of Experts on Consumer Protection Law and Policy\n\n\nVoluntary Peer Review of Competition Law and Policy\n\n\nVoluntary Peer Review of Consumer Protection Law and Policy', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and regulations that are suitable to

Nice! Our chatbot can now answer domain-specific questions in a conversational way.

As an aside, if you don't want to return all the intermediate steps, you can define your retrieval chain like this using a pipe directly into the document chain instead of the final .assign() call:

In [21]:
retrieval_chain_with_only_answer = (
    RunnablePassthrough.assign(
        context=parse_retriever_input | retriever,
    )
    | document_chain
)

retrieval_chain_with_only_answer.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    },
)

"The United Nations Guidelines for Consumer Protection aim to promote fair and transparent business practices, protect consumers' rights, and ensure their safety and well-being. The guidelines cover various aspects of consumer protection, including:\n\n1. Information and Education: Providing accurate and timely information to consumers, as well as promoting education and awareness about consumer rights and responsibilities.\n2. Protection from Unfair Practices: Prohibiting unfair or deceptive business practices, such as false advertising, bait-and-switch tactics, and other forms of consumer exploitation.\n3. Product Safety and Quality: Ensuring that products meet safety and quality standards, and that manufacturers and suppliers are held accountable for any defects or harm caused by their products.\n4. Consumer Redress: Providing mechanisms for consumers to seek redress for any harm or injury caused by a business or product, including compensation for damages and other forms of relief.

## Query transformation
There's one more optimization we'll cover here - in the above example, when we asked a followup question, tell me more about that!, you might notice that the retrieved docs don't directly include information about testing. This is because we're passing tell me more about that! verbatim as a query to the retriever. The output in the retrieval chain is still okay because the document chain retrieval chain can generate an answer based on the chat history, but we could be retrieving more rich and informative documents:

In [22]:
retriever.invoke("Summarise the Guidelines for Consumer Protection")

[Document(page_content='Download:\n\nThe United Nations Guidelines for Consumer Protection\n\t| English | French | Spanish | Arabic | Chinese | Russian |\n\n\n\n\n\n\n\n\n\nRight navigation\n\n\nCompetition and Consumer Protection\n\n\nIntergovernmental Group of Experts on Competition Law and Policy\n\n\nIntergovernmental Group of Experts on Consumer Protection Law and Policy\n\n\nVoluntary Peer Review of Competition Law and Policy\n\n\nVoluntary Peer Review of Consumer Protection Law and Policy', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and regulations that are suitable to their own economic and social and environmental circumstances, as well as promoting international en

In [23]:
retriever.invoke("tell me more about that!")

[Document(page_content='Meetings and events', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and regulations that are suitable to their own economic and social and environmental circumstances, as well as promoting international enforcement coo', 'language': 'en', 'source': 'https://unctad.org/topic/competition-and-consumer-protection/un-guidelines-for-consumer-protection', 'title': 'United Nations guidelines for consumer protection | UNCTAD'}),
 Document(page_content='Meetings and events', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protect

To get around this common problem, let's add a query transformation step that removes references from the input. We'll wrap our old retriever as follows:

In [24]:
# We need a prompt that we can pass into an LLM to generate a transformed search query

# chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

Now let's recreate our earlier chain with this new query_transforming_retriever_chain. Note that this new chain accepts a dict as input and parses a string to pass to the retriever, so we don't have to do additional parsing at the top level:

In [25]:
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

demo_ephemeral_chat_history = ChatMessageHistory()

And finally, let's invoke it!

In [26]:
demo_ephemeral_chat_history.add_user_message("Summarise the Guidelines for Consumer Protection")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])

response 

{'messages': [HumanMessage(content='Summarise the Guidelines for Consumer Protection'),
  AIMessage(content="The United Nations Guidelines for Consumer Protection aim to promote fair and transparent business practices, protect consumers' rights, and ensure their safety and well-being. The guidelines cover various aspects of consumer protection, including:\n\n1. Information and Education: Providing accurate and timely information to consumers, as well as promoting education and awareness about consumer rights and responsibilities.\n2. Protection from Unfair Practices: Prohibiting unfair or deceptive business practices, such as false advertising, bait-and-switch tactics, and other forms of consumer exploitation.\n3. Product Safety and Quality: Ensuring that products meet safety and quality standards, and that manufacturers and suppliers are held accountable for any defects or harm caused by their products.\n4. Consumer Redress: Providing mechanisms for consumers to seek redress for any h

In [27]:
demo_ephemeral_chat_history.add_user_message("tell me more about that!")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='Summarise the Guidelines for Consumer Protection'),
  AIMessage(content="The United Nations Guidelines for Consumer Protection aim to promote fair and transparent business practices, protect consumers' rights, and ensure their safety and well-being. The guidelines cover various aspects of consumer protection, including:\n\n1. Information and Education: Providing accurate and timely information to consumers, as well as promoting education and awareness about consumer rights and responsibilities.\n2. Protection from Unfair Practices: Prohibiting unfair or deceptive business practices, such as false advertising, bait-and-switch tactics, and other forms of consumer exploitation.\n3. Product Safety and Quality: Ensuring that products meet safety and quality standards, and that manufacturers and suppliers are held accountable for any defects or harm caused by their products.\n4. Consumer Redress: Providing mechanisms for consumers to seek redress for any h

In [28]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("thanks for that, so how is the consumers protection guaranteed")

# Invoke the conversational retrieval chain
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})

# Extract and print only the answer
print(response['answer'])


According to the United Nations Guidelines for Consumer Protection, consumers' protection is guaranteed through a combination of measures, including:

1. **Legislation and Regulation**: Governments are encouraged to enact and enforce laws and regulations that protect consumers' rights and interests.
2. **Consumer Education and Awareness**: Governments and businesses are encouraged to educate consumers about their rights and responsibilities, as well as the importance of consumer protection.
3. **Product Safety and Quality**: Businesses are responsible for ensuring that their products meet safety and quality standards, and governments are responsible for enforcing these standards.
4. **Redress Mechanisms**: Governments and businesses are encouraged to establish mechanisms for consumers to seek redress for any harm or injury caused by a business or product.
5. **Complaint Handling**: Businesses are encouraged to establish procedures for handling consumer complaints and to respond promptl

And we now have a chatbot capable of conversational retrieval!

## Next steps
Next is to know how to build a conversational chatbot that can integrate past messages and domain-specific knowledge into its generations. There are  other optimizations we can make around this:

- **Memory management:** This includes a guide on automatically updating chat history, as well as trimming, summarizing, or otherwise modifying long conversations to keep your bot focused.
- **Retrieval:** A deeper dive into using different types of retrieval with the chatbot
- **Tool usage:** How to allows the chatbot to use tools that interact with other APIs and systems.